In [1]:
import carla
import numpy as np
import cv2
import pygame
import torch
import torch.nn as nn
from torchvision import models, transforms
from carla import VehicleLightState

# === Pygame setup ===
pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("CARLA PyTorch Driving")

# --- Connect to CARLA ---
client = carla.Client('localhost', 2000)
client.set_timeout(25.0)
client.load_world("Town05")
world = client.get_world()

# --- Get vehicle and attach camera sensor ---
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
spawn_points = world.get_map().get_spawn_points()

for spawn_point in spawn_points:
    vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)
    if vehicle is not None:
        break

if vehicle is None:
    raise RuntimeError("Could not spawn vehicle due to collisions.")

# --- Set up camera sensor ---
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '320')
camera_bp.set_attribute('image_size_y', '240')
camera_bp.set_attribute('fov', '110')

camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

# === Global variables ===
camera_image = None
signal_indicator = "NONE"

# === Image display helper ===
def process_image(image):
    global camera_image
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    array = array[:, :, :3][:, :, ::-1]
    camera_image_raw = pygame.surfarray.make_surface(array.swapaxes(0, 1))
    camera_image = pygame.transform.scale(camera_image_raw, (800, 600))

# === Dynamic spectator camera follow ===
spectator = world.get_spectator()

def update_spectator():
    transform = vehicle.get_transform()
    forward_vector = transform.get_forward_vector()
    cam_location = transform.location - forward_vector * 8 + carla.Location(z=3)
    cam_rotation = carla.Rotation(pitch=-10, yaw=transform.rotation.yaw)
    spectator.set_transform(carla.Transform(cam_location, cam_rotation))

# === PyTorch Model ===
class DrivingModel(nn.Module):
    def __init__(self):
        super().__init__()
        cnn = models.resnet18(pretrained=False)
        cnn.fc = nn.Identity()
        self.cnn = cnn
        self.fc = nn.Sequential(
            nn.Linear(512 + 1, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, img, signal):
        features = self.cnn(img)
        x = torch.cat((features, signal), dim=1)
        return self.fc(x)

# === Model Setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DrivingModel().to(device)
model.load_state_dict(torch.load("../output/carla_model.pth", map_location=device))
model.eval()

# === Transform ===
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# === Control Loop ===
def drive():
    control = carla.VehicleControl()

    def on_image(image):
        img = np.frombuffer(image.raw_data, dtype=np.uint8)
        img = img.reshape((image.height, image.width, 4))[:, :, :3]
        img_rgb = img[:, :, ::-1]

        # Determine signal
        current_signal = 0
        light_state = vehicle.get_light_state()
        if light_state & carla.VehicleLightState.LeftBlinker:
            current_signal = -1
        elif light_state & carla.VehicleLightState.RightBlinker:
            current_signal = 1

        # Preprocess
        img_tensor = transform(img_rgb).unsqueeze(0).to(device)
        signal_tensor = torch.tensor([[current_signal]], dtype=torch.float32).to(device)

        # Predict
        with torch.no_grad():
            steer = model(img_tensor, signal_tensor).item()
        steer = float(np.clip(steer, -1.0, 1.0))

        # Apply control
        control.steer = steer
        control.throttle = 0.4
        control.brake = 0.0
        vehicle.apply_control(control)

        update_spectator()
        process_image(image)

    camera.listen(lambda image: on_image(image))

# === Main Loop ===
try:
    drive()
    clock = pygame.time.Clock()

    while True:
        keys = pygame.key.get_pressed()

        # Handle signal lights
        if keys[pygame.K_q]:
            vehicle.set_light_state(VehicleLightState.LeftBlinker)
            signal_indicator = "LEFT"
        elif keys[pygame.K_e]:
            vehicle.set_light_state(VehicleLightState.RightBlinker)
            signal_indicator = "RIGHT"
        elif keys[pygame.K_r]:
            vehicle.set_light_state(VehicleLightState.NONE)
            signal_indicator = "NONE"

        # Handle exit
        for event in pygame.event.get():
            if event.type == pygame.QUIT or keys[pygame.K_ESCAPE]:
                raise KeyboardInterrupt

        # Draw image
        font = pygame.font.SysFont("Arial", 36)
        text = font.render("Signal Indicator: " + signal_indicator, True, (255, 255, 255))
        if camera_image:
            screen.blit(camera_image, (0, 0))
            screen.blit(text, (20, 20))
            pygame.display.flip()

        clock.tick(30)

finally:
    camera.stop()
    vehicle.destroy()
    camera.destroy()
    pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\deyru\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\deyru\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


KeyboardInterrupt: 

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.